In [163]:
import numpy as np
import tensorflow as tf
import random as rn
import os
import sys
from functools import reduce

# os.environ['PYTHONHASHSEED'] = '0'
os.environ['CUDA_VISIBLE_DEVICES'] = ''
# # random seed for numpy
# np.random.seed(37)
# # random seed for python
# rn.seed(1254)
# # random seed for tensorflow
# tf.random.set_seed(89)
from keras import backend as K


# session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

#Force Tensorflow to use a single thread
# sess = tf.compat.v1.Session(graph=tf.compat.v1.reset_default_graph(), config=session_conf)
# sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
# K.set_session(sess)


import pandas as pd
import pyarrow.parquet as pq

import csv
import glob
# from itertools import chain
from keras import models
from keras import layers
import keras 

from sklearn.model_selection import train_test_split

from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense

from collections import Counter
# import matplotlib.pyplot as plt
from keras.callbacks import History


import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline      # for warping
from transforms3d.axangles import axangle2mat  # for rotation


np.set_printoptions(threshold=sys.maxsize)

In [98]:
# given a path like "../trial/S01/ACC_01.parquet" return:
# the number of ms that a file needs in order to do the downsampling;
# and return the type among:
# - 0 (EDA)
# - 1 (BVP)
# - 2 (ACC)
# - 3 (TEM) 

# If the file is incorrect return -1
def type_file(name):
    name_file = name.split('/')[3][:3]
    if name_file == 'EDA':
        return 0, 0
    elif name_file == 'BVP':
        return '250ms', 1
    elif name_file == 'ACC':
        return '250ms', 2
    elif name_file == 'TEM':
        return 0, 5
    return -1

In [99]:
# Function to determine the most frequent element in a list:
# [0,0,0,1,1] return 0
# [0,0,1,1,1] return 1

def most_frequent(List):
    counter = 0
    num = List[0]

    for i in List:
        curr_frequency = List.count(i)
        if(curr_frequency> counter):
            counter = curr_frequency
            num = i
    return num

# listt = [0,1,0,1]
# print(most_frequent(listt))

In [100]:
def append_data_and_fill_missing_nan(ARRAY, df, value):
    for i in range(0, df.shape[0]):
        ARRAY.append(df[value][i])

    if (len(ARRAY) % 2400) != 0:
        rounding = int(len(ARRAY)//2400 + (len(ARRAY) % 2400 > 0))
        number_cells = rounding * 2400
        missing_zeros = [np.nan] * (number_cells - len(ARRAY))
        ARRAY = ARRAY + missing_zeros
    return ARRAY
    
    

In [101]:
def find_sleep_label(LABEL, EDA, df):
    i = 0
    list_sleep = []
#     create the list_sleep with all the values of the colums 'Sleep'
    for j in range(0, df.shape[0]):
        value = df['Sleep'][j].astype(np.float64)
        list_sleep.append(value)

#   for each 2400 window choose if 0 or 1, and append it to ARRAR_LABEL
    while i in range(0, len(list_sleep)- 2399):
        list_sleep_window = list_sleep[i:(i+2400)]
        label_sleep = most_frequent(list_sleep_window)
        LABEL.append(label_sleep)
        i += 2400
        
#     if EDA len is greather than ARRAY_LABEL, uniform it to the same length
    if (len(LABEL) < int(len(EDA)/2400)):
        missing_zeros = [np.nan]*(int(len(EDA)/2400)-len(LABEL))
        LABEL = LABEL + missing_zeros
    return LABEL
        

In [102]:
def add_zeros_to_uniform_size_array(ARRAY, missing_zeros):
    ARRAY = [np.nan]* missing_zeros
    return ARRAY

In [103]:
def remove_nan_arrays(ARRAY, ARR2, ARR3, ARR4, ARR5, ARR6, LABEL):
    array_zeros = np.zeros(2400)
    
    array_boolean = np.isnan(ARRAY).any(axis=1)

    # iterate over each row.
    # if delete a "true" then continue in a while untill it goes 1 step forward
    i = 0
    while i < len(array_boolean):
        while array_boolean[i]:
            ARRAY = np.delete(ARRAY, i, 0)
            ARR2 = np.delete(ARR2, i, 0)
            ARR3 = np.delete(ARR3, i, 0)
            ARR4 = np.delete(ARR4, i, 0)
            ARR5 = np.delete(ARR5, i, 0)
            ARR6 = np.delete(ARR6, i, 0)
            LABEL = np.delete(LABEL, i, 0)
            array_boolean = np.delete(array_boolean, i)
        i += 1
    
    ARRAY = ARRAY[:-1]
    ARR2 = ARR2[:-1]
    ARR3 = ARR3[:-1]
    ARR4 = ARR4[:-1]
    ARR5 = ARR5[:-1]
    ARR6 = ARR6[:-1]
    LABEL = LABEL[:-1]
    return ARRAY, ARR2, ARR3, ARR4, ARR5, ARR6, LABEL

In [104]:
# from array 1D, i create an array 2D with the last row as 0.
def reshape_arrays(EDA):
    EDA = np.array(EDA)
#     print("eda before: ",EDA.shape)
    EDA = EDA.reshape(1,len(EDA)//2400, 2400)
    array_zeros = np.zeros(2400)
    EDA = EDA[0]
    EDA = np.concatenate((EDA, [array_zeros]), axis=0)
    
#     print("eda after: ",EDA.shape)
    return EDA

In [105]:
# from array 1D, i append a 0 in the last row.
def reshape_label(LABEL):
    LABEL = np.array(LABEL)
#     print("LABEL before: ", LABEL.shape)
#     LABEL = LABEL.reshape(1,len(LABEL), 1)
#     array_zeros = np.zeros(2400)
#     LABEL = LABEL[0]
    LABEL = np.concatenate((LABEL, [0]), axis=0)
    
#     print("LABEL after: ",LABEL.shape)
    return LABEL

In [237]:
# Call function create_train_set_train_label so that we have the two zero tensors.
# for each file read the data, downsampling the data and fill the tensors.

def read_data(files):
    EDA, BVP, ACC_X, ACC_Y, ACC_Z, TEM, LABEL = [],[],[],[],[],[],[]


    # READ ALL THE FILES and DOWNSAMPLING
    for name_file in files:
#         print(name_file)
        rounding, number_cells, missing_zeros = 0, 0, []
        # DataFrame
        table = pd.read_parquet(name_file, engine='pyarrow')
        # creating DataFrame
        df = pd.DataFrame(table)
        # converting timestamp
        timestamp_col = pd.to_datetime(df['timestamp'], unit='s')
        df['timestamp'] = timestamp_col
        # get if file it's among EDA, BVP, ACC, ST, otherwise return error
        arr_type = type_file(name_file)
        if arr_type == -1:
            print("Error in the file name")
            return -1

        # RESAMPLE the tensor
        # 0 means it's already 4Hz, otherwise resample with '250ms'
        if arr_type[0] != 0:
            df = df.resample(arr_type[0], on='timestamp').mean()
            df = df.reset_index()
        
        # remove the          
        
#         print(df.shape)
       # CREATES THE ARRAYS
        # create linear arrays as sequences of elements. for each file add several zeros to complete a window of 2400     
        if arr_type[1] == 0:
            EDA = append_data_and_fill_missing_nan(EDA, df, 'value')
            LABEL = find_sleep_label(LABEL, EDA, df)
                
        elif arr_type[1] == 1:
            BVP = append_data_and_fill_missing_nan(BVP, df, 'value')

        elif arr_type[1] == 2:
            ACC_X = append_data_and_fill_missing_nan(ACC_X, df, 'X')
            ACC_Y = append_data_and_fill_missing_nan(ACC_Y, df, 'Y')
            ACC_Z = append_data_and_fill_missing_nan(ACC_Z, df, 'Z')
            
        elif arr_type[1] == 5:
            TEM = append_data_and_fill_missing_nan(TEM, df, 'value')
        
        
#     print("LEN LABEL",len(LABEL))
#     if each file has the same length it doesn make sense to check it. only in between files fill the row of zeros
#     and then start from the next row a new file
#     if the files are not the same quantity, it's better to throw error?
    uniform_len = max(len(EDA),len(BVP),len(ACC_X),len(ACC_Y),len(ACC_Z),len(TEM))    
#     print(len(EDA), ' ',len(BVP), ' ',len(ACC_X), ' ',len(ACC_Y), ' ',len(ACC_Z), ' ',len(TEM) )
#     check train_set
    if len(EDA) != uniform_len:
        EDA = EDA + [np.nan]* (uniform_len - len(EDA))
    if len(BVP) != uniform_len:
        BVP = BVP + [np.nan]* (uniform_len - len(BVP))
    if len(ACC_X) != uniform_len:
        ACC_X = ACC_X + [np.nan]* (uniform_len - len(ACC_X))
    if len(ACC_Y) != uniform_len:
        ACC_Y = ACC_Y + [np.nan]* (uniform_len - len(ACC_Y))
    if len(ACC_Z) != uniform_len:
        ACC_Z = ACC_Z + [np.nan]* (uniform_len - len(ACC_Z))
    if len(TEM) != uniform_len:
        TEM = TEM + [np.nan]* (uniform_len - len(TEM))
        
#     print(len(EDA_LABEL), ' ',len(BVP_LABEL), ' ',len(ACC_X_LABEL), ' ',len(ACC_Y_LABEL), ' ',len(ACC_Z_LABEL), ' ',len(TEM_LABEL) )
    
    
    # convert the list to a numpy array
#     removing rows with nan
    EDA_np, BVP_np, TEM_np = np.array(EDA),np.array(BVP),np.array(TEM)
    ACC_X_np, ACC_Y_np, ACC_Z_np = np.array(ACC_X),np.array(ACC_Y),np.array(ACC_Z)
    LABEL_np = np.array(LABEL)

    
    EDA_np = reshape_arrays(EDA)
    BVP_np = reshape_arrays(BVP)
    ACC_X_np = reshape_arrays(ACC_X)
    ACC_Y_np = reshape_arrays(ACC_Y)
    ACC_Z_np = reshape_arrays(ACC_Z)
    TEM_np = reshape_arrays(TEM)
    LABEL_np =  reshape_label(LABEL)

    

    EDA_np, BVP_np, ACC_X_np, ACC_Y_np, ACC_Z_np, TEM_np,LABEL_np = remove_nan_arrays(EDA_np, BVP_np, ACC_X_np, ACC_Y_np, ACC_Z_np, TEM_np, LABEL_np)
# each file has different nan rows
    BVP_np, EDA_np, ACC_X_np, ACC_Y_np, ACC_Z_np, TEM_np,LABEL_np = remove_nan_arrays(BVP_np, EDA_np, ACC_X_np, ACC_Y_np, ACC_Z_np, TEM_np,LABEL_np)
    ACC_X_np, EDA_np, BVP_np, ACC_Y_np, ACC_Z_np, TEM_np,LABEL_np = remove_nan_arrays(ACC_X_np, EDA_np, BVP_np, ACC_Y_np, ACC_Z_np, TEM_np,LABEL_np)
    ACC_Y_np, EDA_np, BVP_np, ACC_X_np, ACC_Z_np, TEM_np,LABEL_np = remove_nan_arrays(ACC_Y_np, EDA_np, BVP_np, ACC_X_np, ACC_Y_np, TEM_np,LABEL_np)
    ACC_Z_np, EDA_np, BVP_np, ACC_X_np, ACC_Y_np, TEM_np,LABEL_np = remove_nan_arrays(ACC_Z_np, EDA_np, BVP_np, ACC_X_np, ACC_Y_np, TEM_np,LABEL_np)
    TEM_np, EDA_np, BVP_np, ACC_X_np, ACC_Y_np, ACC_Z_np,LABEL_np = remove_nan_arrays(TEM_np, EDA_np, BVP_np, ACC_X_np, ACC_Y_np, ACC_Z_np,LABEL_np)



    EDA = EDA_np.flatten().tolist()
    BVP = BVP_np.flatten().tolist()
    ACC_X = ACC_X_np.flatten().tolist()
    ACC_Y = ACC_Y_np.flatten().tolist()
    ACC_Z = ACC_Z_np.flatten().tolist()
    TEM = TEM_np.flatten().tolist()
    
    
    train_set = np.array([EDA, BVP, ACC_X, ACC_Y, ACC_Z, TEM],dtype=np.float64)
    train_label = np.array([LABEL_np], dtype=np.float64)
    

    # Reshape the tensor train set, counting the number of rows dividing by 2400
    uniform_len = max(len(EDA),len(BVP),len(ACC_X),len(ACC_Y),len(ACC_Z),len(TEM))    
    row_training_set = uniform_len // 2400
    if (uniform_len % 2400) != 0:
        return "Error in missing_zeros"
    
    train_set = train_set.reshape(6,row_training_set,2400)
    
    print("before augmentation")
    print(train_set.shape)
    print(train_label.shape)
    

    return train_set, train_label




In [238]:
def scale(X, sigma=0.1):
    scalingFactor = np.random.normal(loc=1.0, scale=sigma, size=(1, X.shape[1]))  # shape=(1,3)
    myNoise = np.matmul(np.ones((X.shape[0], 1)), scalingFactor)
    return X * myNoise


In [239]:
# sigma = 0.05
# Hyperparameters :  sigma = standard devitation (STD) of the noise
def jitter(X, sigma=0.05):
    myNoise = np.random.normal(loc=0, scale=sigma, size=X.shape)
    print("jitter")
    return X + myNoise


In [240]:
def GenerateRandomCurves(X, sigma=0.2, knot=4):
    xx = (np.ones((X.shape[1], 1)) * (np.arange(0, X.shape[0], (X.shape[0] - 1) / (knot + 1)))).transpose()
    yy = np.random.normal(loc=1.0, scale=sigma, size=(knot + 2, X.shape[1]))
    x_range = np.arange(X.shape[0])
    random_curves = []
    for i in range(X.shape[-1]):
        cs = CubicSpline(xx[:, i], yy[:, i])
        random_curves.append(cs(x_range))
    return np.array(random_curves).transpose()

def DA_MagWarp(X, sigma=0.2):
    return X * GenerateRandomCurves(X, sigma)


In [241]:
def create_datasets(start_session, end_session):
#     create list of folders
    listt = os.listdir("../Sessions/")
#     print(listt)
#   how many users? for now only 1
    list_dir = sorted(listt)[0:1]
#     list_dir = sorted(listt)
    list_path_dir = []
    for elem in list_dir:
        list_path_dir.append("../Sessions/" + elem)

#     read files in the folder and create a list of files 
    list_ordered_files = []
    
    for folder in list_path_dir:
#         print(folder)
        list_files = []
        for infile in os.listdir(folder):
            list_files.append(infile)
        list_files = sorted(list_files)
#         print(list_files)

    #     divide the list into 4 sections: acc, bvp, eda, temp
    #     take one from each and construct a list 
    #     the order is really important while reading files
        
        acc = []
        bvp = []
        eda = []
        tem = []
        for elem in (list_files):
            if elem[:3] == "ACC":
                acc.append(elem)
            elif elem[:3] == "BVP":
                bvp.append(elem)
            elif elem[:3] == "EDA":
                eda.append(elem)
            elif elem[:3] == "TEM":
                tem.append(elem)

        acc = acc[start_session:end_session]
        bvp = bvp[start_session:end_session]
        eda = eda[start_session:end_session]
        tem = tem[start_session:end_session]
    #     create a list with that order: [acc1, bvp1, eda1, tem1, acc2, bvp2, eda2, tem2,...]
        for i in range(len(acc)):
            list_ordered_files.append(folder + "/" + acc[i])
            list_ordered_files.append(folder + "/" + bvp[i])
            list_ordered_files.append(folder + "/" + eda[i])
            list_ordered_files.append(folder + "/" + tem[i])
#     print(list_ordered_files)
        
    
    return read_data(list_ordered_files)

In [244]:
def augment_data(train_set, train_label):
    print(train_set.shape)
#     i have to compat the ACC_X, ACC_Y, ACC_Z to ACC with formula √(x*x + y*y + z*z)
    acc_x_squared = np.multiply(train_set[2], train_set[2])
    acc_y_squared = np.multiply(train_set[3], train_set[3])
    acc_z_squared = np.multiply(train_set[4], train_set[4])
    acc_xyz_sum = acc_x_squared + acc_y_squared + acc_z_squared
    acc = np.sqrt(acc_xyz_sum)
    
#     train_set = np.array([train_set[0], train_set[1], acc, train_set[5]])
    print(train_set.shape)
    
#     START AUGMENTING
    ARR,LABEL = [], []
    
#     print(train_set.shape)
#     print(train_label.shape)
    # select random indices
    number_of_rows = int(train_set.shape[1] * 0.5)

#     random indices has to be the same for every dimension so that the label can be accurate
    random_indices = np.sort(np.random.choice(train_set.shape[1]-1, size=int(number_of_rows), replace=False))
    #     iterate over every dimension
#     augment every dimension of the tensor so that at the end we have a tensor augmented
    for train_set_one in train_set:
    # take partial array with random indices
        train_set_one = train_set_one[random_indices,:]
    # perform jittering on the partial array
        train_set_one = train_set_one.transpose()
#         SCALE AUGMENT
        train_set_one = scale(train_set_one)
        train_set_one = train_set_one.transpose()
        
#     create an array ARR only of augmented data
        ARR = [*ARR, train_set_one]

    ARR = np.array(ARR)
    # take the label and add them as the label for the new augmented data
    LABEL = np.array(train_label[:,random_indices])

#     we have ARR which is of shape (6, row, col) with the augmented data
#     and train_set which is of shape (6, row, col) with the non augmented data
    train_set_augmented = np.concatenate((train_set, ARR), axis = 1)
    
    label_set_augmented = np.concatenate((train_label, LABEL), axis=1)
#     print(label_set_augmented.shape)
#     print(train_set_augmented.shape)
    
    return train_set_augmented,label_set_augmented

# augment_data(train_set, train_label)

In [245]:
def get_augmented_data(train_set, train_label):
    print("start augmenting")
    train_set_augmented, label_set_augmented = augment_data(train_set, train_label)
#     reshape the label for the networking part
    train_label = train_label.reshape(train_label.shape[1], 1)
    label_set_augmented = label_set_augmented.reshape(label_set_augmented.shape[1], 1)
    
    print(train_set.shape)
    print(train_label.shape)
    print(train_set_augmented.shape)
    print(label_set_augmented.shape)
    print("end augmenting")
    
    return train_set, train_label, train_set_augmented, label_set_augmented


In [ ]:
# 4 is the number of sessions per user
# train_set, train_label = read_files(0,4)
# from 0 session to 5 session per user
train_set, train_label =  create_datasets(0,4)
train_set, train_label, train_set_augmented, label_set_augmented = get_augmented_data(train_set, train_label)

In [123]:
# create the test set
test_set, test_label = create_datasets(5,6)
test_label = test_label.reshape(test_label.shape[1], 1)

before augmentation
(6, 721, 2400)
(1, 721)


In [ ]:
def compat_accelerometer(train_set, train_label):
    train_acc = 
    

In [23]:
# EDA EDA EDA EDA EDA EDA EDA EDA EDA EDA EDA EDA EDA EDA EDA EDA EDA EDA EDA EDA EDA EDA EDA EDA EDA EDA EDA EDA

In [222]:
seed_value= 0
os.environ['PYTHONHASHSEED']=str(seed_value)
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ""
# rn.seed(seed_value)
# np.random.seed(seed_value)
tf.random.set_seed(seed_value)



def network_model():
    model = Sequential()
    model.add(Dense(32, activation='relu', input_shape=(2400,)))
#     model.add(Dense(16, activation='relu'))
#     model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    opt = Adam(learning_rate=0.0001)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

    
    
#     ORIGINAL SET
    train_set_eda = train_set[0]
    test_set_eda = test_set[0]
    print(train_set_eda.shape)
#     print(test_label.shape)
    print(test_set_eda.shape)
    
    avg_accuracy = 0
    for i in range(0, 1):
        histor = model.fit(train_set_eda, train_label, epochs=15, batch_size=128,shuffle=False, verbose=0)
        scores = model.evaluate(test_set_eda, test_label,verbose=0)
        avg_accuracy += scores[1]
#         print('Test loss: {} - Accuracy: {}'.format(*scores))
#         avg = reduce(lambda x, y: x + y, histor.history['accuracy']) / len(histor.history['accuracy'])
#         print(avg)
    print("Accuracy of 50 fit of original set: ", avg_accuracy/1)
    
#     AUGMENTATION
    train_set_eda_augment = train_set_augmented[0]
#     print(train_set_eda_augment.shape)
#     print(label_set_augmented.shape)

    avg_accuracy = 0
    for i in range(0, 1):
        histor = model.fit(train_set_eda_augment, label_set_augmented, epochs=15, batch_size=128,shuffle=False,verbose=0)
        scores = model.evaluate(test_set_eda, test_label,verbose=0)
        avg_accuracy += scores[1]
#         print('Test loss: {} - Accuracy: {}'.format(*scores))
#         avg = reduce(lambda x, y: x + y, histor.history['accuracy']) / len(histor.history['accuracy'])
#         print(avg)
    print("Accuracy of 50 fit of EDA set: ", avg_accuracy/1)
    print("-----END-----")

network_model()

    


(2689, 2400)
(721, 2400)
Accuracy of 50 fit of original set:  0.5312066674232483
Accuracy of 50 fit of EDA set:  0.6352288722991943
-----END-----


In [ ]:
# BVC BVC BVC BVC BVC BVC BVC BVC BVC BVC BVC BVC BVC BVC BVC BVC BVC BVC BVC BVC BVC BVC BVC BVC BVC BVC BVC BVC BVC

In [225]:
seed_value= 0
os.environ['PYTHONHASHSEED']=str(seed_value)
# rn.seed(seed_value)
# np.random.seed(seed_value)
tf.random.set_seed(seed_value)


def network_model():
    model = Sequential()
    model.add(Dense(32, activation='relu', input_shape=(2400,)))
#     model.add(Dense(16, activation='relu'))
#     model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    opt = Adam(learning_rate=0.0001)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

    
    
#     ORIGINAL SET
    train_set_eda = train_set[1]
    test_set_eda = test_set[1]
    print(train_set_eda.shape)
#     print(test_label.shape)
    print(test_set_eda.shape)
    
    avg_accuracy = 0
    for i in range(0, 1):
        histor = model.fit(train_set_eda, train_label, epochs=15, batch_size=128,shuffle=False, verbose=0)
        scores = model.evaluate(test_set_eda, test_label,verbose=0)
        avg_accuracy += scores[1]
#         print('Test loss: {} - Accuracy: {}'.format(*scores))
#         avg = reduce(lambda x, y: x + y, histor.history['accuracy']) / len(histor.history['accuracy'])
#         print(avg)
    print("Accuracy of 50 fit of original set: ", avg_accuracy/1)
    
#     AUGMENTATION
    train_set_eda_augment = train_set_augmented[1]
#     print(train_set_eda_augment.shape)
#     print(label_set_augmented.shape)

    avg_accuracy = 0
    for i in range(0, 1):
        histor = model.fit(train_set_eda_augment, label_set_augmented, epochs=15, batch_size=128,shuffle=False,verbose=0)
        scores = model.evaluate(test_set_eda, test_label,verbose=0)
        avg_accuracy += scores[1]
#         print('Test loss: {} - Accuracy: {}'.format(*scores))
#         avg = reduce(lambda x, y: x + y, histor.history['accuracy']) / len(histor.history['accuracy'])
#         print(avg)
    print("Accuracy of 50 fit of EDA set: ", avg_accuracy/1)
    print("-----END-----")

network_model()



(2689, 2400)
(721, 2400)
Accuracy of 50 fit of original set:  0.484049916267395
Accuracy of 50 fit of EDA set:  0.4882108271121979
-----END-----


In [ ]:
# TEMP TEMP TEMP TEMP TEMP TEMP TEMP TEMP TEMP TEMP TEMP TEMP TEMP TEMP TEMP TEMP TEMP TEMP TEMP TEMP TEMP TEMP TEMP

In [226]:
seed_value= 0
os.environ['PYTHONHASHSEED']=str(seed_value)
rn.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)


def network_model():
    model = Sequential()
    model.add(Dense(32, activation='relu', input_shape=(2400,)))
#     model.add(Dense(16, activation='relu'))
#     model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    opt = Adam(learning_rate=0.0001)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

    
    
#     ORIGINAL SET
    train_set_eda = train_set[5]
    test_set_eda = test_set[5]
    print(train_set_eda.shape)
#     print(test_label.shape)
    print(test_set_eda.shape)
    
    avg_accuracy = 0
    for i in range(0, 1):
        histor = model.fit(train_set_eda, train_label, epochs=15, batch_size=128,shuffle=False, verbose=0)
        scores = model.evaluate(test_set_eda, test_label,verbose=0)
        avg_accuracy += scores[1]
#         print('Test loss: {} - Accuracy: {}'.format(*scores))
#         avg = reduce(lambda x, y: x + y, histor.history['accuracy']) / len(histor.history['accuracy'])
#         print(avg)
    print("Accuracy of 50 fit of original set: ", avg_accuracy/1)
    
#     AUGMENTATION
    train_set_eda_augment = train_set_augmented[5]
#     print(train_set_eda_augment.shape)
#     print(label_set_augmented.shape)

    avg_accuracy = 0
    for i in range(0, 1):
        histor = model.fit(train_set_eda_augment, label_set_augmented, epochs=15, batch_size=128,shuffle=False,verbose=0)
        scores = model.evaluate(test_set_eda, test_label,verbose=0)
        avg_accuracy += scores[1]
#         print('Test loss: {} - Accuracy: {}'.format(*scores))
#         avg = reduce(lambda x, y: x + y, histor.history['accuracy']) / len(histor.history['accuracy'])
#         print(avg)
    print("Accuracy of 50 fit of EDA set: ", avg_accuracy/1)
    print("-----END-----")

network_model()



(2689, 2400)
(721, 2400)
Accuracy of 50 fit of original set:  0.5284327268600464
Accuracy of 50 fit of EDA set:  0.47295424342155457
-----END-----


In [ ]:
# 3600 samples (Time) * 3 features (X,Y,Z)
# take two dimensions array

# [[acc_x_0], [acc_y_0], [acc_z_0]]
eda = train_set[0][0]
bvp = train_set[1][0]
acc_x_0 = train_set[2][0]
acc_y_0 = train_set[3][0]
acc_z_0 = train_set[4][0]
tem = train_set[5][0]


array_to_plot = np.array([acc_x_0,acc_y_0,acc_z_0], dtype=np.float64)
# plt.plot(array_to_plot)

# plt.plot(acc_x_0, color='g')
# plt.plot(acc_y_0, color='r')
# plt.plot(acc_z_0, color='b')
# plt.title("A window of 10 min Acceleration data")
# plt.axis([0,2400,-100,100])
# plt.xlabel("time")
# plt.ylabel("value acc")
# plt.show()


plt.plot(eda, color='g')
plt.title("A window of 10 min Electrodermal Activity data")
plt.axis([0,2400,-0.5,1.5])
plt.xlabel("time")
plt.ylabel("value acc")
plt.show()

# plt.plot(bvp, color='g')
# plt.title("A window of 10 min Blood Volume Pulse data")
# plt.axis([0,2400,-450, 450])
# plt.xlabel("time")
# plt.ylabel("value acc")
# plt.show()

# plt.plot(tem, color='g')
# plt.title("A window of 10 min Skin Temperature data")
# plt.axis([0,2400,28, 29.5])
# plt.xlabel("time")
# plt.ylabel("value acc")
# plt.show()




sigma = 0.05

def DA_Jitter(X, sigma=0.05):
    myNoise = np.random.normal(loc=0, scale=sigma, size=X.shape)
    return X+myNoise


fig = plt.figure(figsize=(15,4))
for ii in range(8):
    ax = fig.add_subplot(2,4,ii+1)
    ax.plot(DA_Jitter(eda, sigma))
    ax.set_xlim([0,2400])
    ax.set_ylim([-0.5,1.5])



In [232]:
a = np.array([[[1,1,1], [2,2,2]], [[3,3,3],[4,4,4]]])
b = np.array([[[5,5,5], [6,6,6]], [[7,7,7],[0,0,0]]])
print(a)
print()
print(b)
print()
# print(np.multiply(a, b))
print(a+b)

[[[1 1 1]
  [2 2 2]]

 [[3 3 3]
  [4 4 4]]]

[[[5 5 5]
  [6 6 6]]

 [[7 7 7]
  [0 0 0]]]

[[[ 6  6  6]
  [ 8  8  8]]

 [[10 10 10]
  [ 4  4  4]]]
